# Boolean Search

### Description
To support queries of matching keywords is a “must-have” function in databases. A query term with boolean operations, such as “國際 and 籃球", is very useful to quickly identify required content in databases.

In [4]:
import argparse
import csv
import time

## [Step1] Build Search Engine

### (Method 1) Use Jieba Package 

剛開始試過使用"結巴"斷詞系統，將source file的資料進行斷詞並進行index

e.g. 2.  美國報告未提證據 俄：操縱大選是無中生有 -->  "美國" "報告" "未提" "證據" "俄" ...

使用斷詞結果當成 key 建立 dictionary

e.g. `[美國] = [2]` `[報告] = [2]` `[未提] = [2]` ... 

### Analysis
因為結巴斷詞需要一段時間，並且根據單詞為 key 會導致字典內容太大，搜尋的速度下降 (斷詞搜尋的時間在測試的 data set 上約為 **5.3s**)

### (Method2) Not Use Jieba Package

為了降低搜尋所需的時間，我更改了建立 search engine 的方法。

e.g. 

2 美國報告未提證據 俄：操縱大選是無中生有  -->  `[美國報告未提證據 俄：操縱大選是無中生有] = [2]`

3 新年搶優惠　通路Apple商品加碼送  --> `[新年搶優惠　通路Apple商品加碼送] = [3]`

In [8]:
dict = {}
with open("source.csv", 'r') as csvfile:
    data = csv.reader(csvfile)
    for row in data:
        try: 
            dict[row[1]].append(row[0])
        except:
            dict[row[1]] = [row[0]]

### Analysis
字典大小縮小，但是每一筆資料的 key 長度增加，因此需要搭配 python 的 `in operator` 進行搜尋 (搜尋的時間在測試的 data set 上約為 **4.5s**)

---------------------

## [Step 2] Query

### Basic
透過建立好的 search engine，查詢對應的 index list

e.g.
川普 and 美國 (`[川普] = [1, 2, 3, 10, 14, 19 ...]` `[美國] = [2, 3, 14, 19, 23, 25 ...]`)

再針對取得的 list 進行操作 `[1, 2, 3, 10, 14, 19 ...] and [2, 3, 14, 19, 23, 25 ...] = [2, 3, ...]`

### Analysis
query 的時間會隨著數量而線性增加 --> 實作 `cache`，減少重複 query 的時間

### Approvement

每次進行 query 時，先從 `cache` 中尋找是否已經有查詢過，有的話則直接取出使用，若沒有再進行上述步驟，並把結果存入 cache 以便下次查詢

In [ ]:
try:
    a = cache[result[0]]
except:
    a = [value for key, value in dict.items() if result[0] in key]
    a = sum(a, [])
    cache[result[0]] = a

try:
    b = cache[result[2]]
except:
    b = [value for key, value in dict.items() if result[2] in key]                       
    b = sum(b, [])
    cache[result[2]] = b

if(result[1] == 'and'):
    ans = list(set(a) & set(b))
elif(result[1] == 'or'):
    ans = list(set(a) | set(b))
elif(result[1] == 'not'):
    ans = list(set(a) - (set(a) & set(b)))

----------------

## Source Code

In [12]:
# encoding=UTF-8
#import jieba
import argparse
import csv
import time

if __name__ == '__main__':
    # You should not modify this part.    
    
    # Please implement your algorithm below
    start_time = time.time()
    
    # TODO load source data, build search engine
    dict = {}
    cache = {}
    with open("source.csv", 'r') as csvfile:
        data = csv.reader(csvfile)
        for row in data:
            try: 
                dict[row[1]].append(row[0])
            except:
                dict[row[1]] = [row[0]]
    print("build search engine --- %s seconds ---" % (time.time() - start_time))        
    start_time = time.time()
    
    # TODO compute query result
    #jieba.set_dictionary('dict.txt.big.txt')
    querys = []    
    with open("query.txt", 'r') as csvfile:
        data = csv.reader(csvfile)
        querys = sum(list(data), [])
    
    # TODO output result
    with open("output.txt", 'w') as output_file:
        for i in range(len(querys)):
            result = querys[i].split()
            
            if(len(result) == 3):
                try:
                    a = cache[result[0]]
                except:
                    a = [value for key, value in dict.items() if result[0] in key]
                    a = sum(a, [])
                    cache[result[0]] = a
                
                try:
                    b = cache[result[2]]
                except:
                    b = [value for key, value in dict.items() if result[2] in key]                       
                    b = sum(b, [])
                    cache[result[2]] = b
    
                if(result[1] == 'and'):
                    ans = list(set(a) & set(b))
                elif(result[1] == 'or'):
                    ans = list(set(a) | set(b))
                elif(result[1] == 'not'):
                    ans = list(set(a) - (set(a) & set(b)))
            
            elif(len(result) == 5):
                try:
                    a = cache[result[0]]
                except:
                    a = [value for key, value in dict.items() if result[0] in key]
                    a = sum(a, [])
                    cache[result[0]] = a
                
                try:
                    b = cache[result[2]]
                except:
                    b = [value for key, value in dict.items() if result[2] in key]                       
                    b = sum(b, [])
                    cache[result[2]] = b
                    
                try:
                    c = cache[result[4]]
                except:
                    c = [value for key, value in dict.items() if result[4] in key]                       
                    c = sum(c, [])
                    cache[result[4]] = c
                
                if(result[1] == 'and'):
                    tmp = set(a) & set(b)
                elif(result[1] == 'or'):
                    tmp = set(a) | set(b)
                elif(result[1] == 'not'):
                    tmp = set(a) - (set(a) & set(b))
            
                if(result[3] == 'and'):
                    ans = tmp & set(c)
                elif(result[3] == 'or'):
                    ans = tmp | set(c)
                elif(result[3] == 'not'):
                    ans = tmp - (tmp & set(c))

            
            if len(ans) == 0:
                output_file.write("0")
            else:
                output_file.write(str(','.join(sorted(ans,  key=lambda x: int(x)))))
    
            if i != len(querys) - 1:
                output_file.write("\n")
                
    print("query --- %s seconds ---" % (time.time() - start_time))

build search engine --- 0.15063905715942383 seconds ---
query --- 0.17416787147521973 seconds ---
